<a href="https://colab.research.google.com/github/aokozlova/02_eurovision/blob/main/ESC_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

from time import sleep
import re
import random

In [4]:
base_url = 'https://eurovision.tv'

#Getting winners and links for the events

In [ ]:
#to collect a table from the official website
#number_of_links - number of links in the table (go and look)
#collect_until - index of a column until collect the links
def collect_table(r, table_selector, number_of_links, collect_until=None):
  soup = BeautifulSoup(r.text,'html.parser')
  table = soup.select_one(table_selector)
  columns = [th.text for th in table.find_all('th')]
  columns = columns +  ['link' + str(i) for i in range(0, number_of_links)]

  data = []
  rows = table.tbody.find_all('tr')
  for row in rows:
    cells = row.find_all('td')
    cells = [c.text.strip() for c in cells] + [c.find('a')['href'] for c in cells[:collect_until] if c.find('a')]
    data.append([c for c in cells])
  df = pd.DataFrame(data = data, columns = columns)
  return df

In [ ]:
winners_url = "https://eurovision.tv/history"
number_of_links = 5 #on the page
all_data = []

r = requests.get(url=winners_url, headers={'User-Agent': 'Mozilla/5.0'})
if r.ok:
  soup = BeautifulSoup(r.text, 'html.parser')

  pages = soup.select_one('.pager__items.js-pager__items').find_all('li')
  pages = set([li.find('a')['href'] for li in pages])
  

  for page in pages:
    r = requests.get(url = winners_url + page, headers = {'User-Agent': 'Mozilla/5.0'})
    if r.ok:
      try:
        df = collect_table(r, '.cols-7',number_of_links)
        all_data.append(df)
        print(page,' collected')
      except NameError:
        print('error', page)
    else:
      print(r.status_code, winners_url + page)

    

?search=&page=0  collected
?search=&page=4  collected
?search=&page=3  collected
?search=&page=2  collected
?search=&page=1  collected


In [ ]:
winners = pd.concat(all_data, ignore_index=True)
winners.head()

,Year,City,Winners,Participant,Song,Points,,link0,link1,link2,link3,link4
0,2022,Turin,Ukraine,Kalush Orchestra,Stefania,631,More,/event/turin-2022,/country/ukraine,/participant/kalush-orchestra-22,https://youtu.be/UiEGVYOruLk,/event/turin-2022
1,2021,Rotterdam,Italy,Måneskin,Zitti E Buoni,524,More,/event/rotterdam-2021,/country/italy,/participant/maneskin-2021,https://youtu.be/9mL6Cmkg2_A,/event/rotterdam-2021
2,2019,Tel Aviv,Netherlands,Duncan Laurence,Arcade,498,More,/event/tel-aviv-2019,/country/the-netherlands,/participant/duncan-laurence,https://youtu.be/R3D-r4ogr7s,/event/tel-aviv-2019
3,2018,Lisbon,Israel,Netta,TOY,529,More,/event/lisbon-2018,/country/israel,/participant/netta-barzilai,https://youtu.be/fnB783uNjN0,/event/lisbon-2018
4,2017,Kyiv,Portugal,Salvador Sobral,Amar Pelos Dois,758,More,/event/kyiv-2017,/country/portugal,/participant/salvador-sobral,https://youtu.be/ymFVfzu-2mw,/event/kyiv-2017


In [ ]:
winners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Year         69 non-null     object
 1   City         69 non-null     object
 2   Winners      69 non-null     object
 3   Participant  69 non-null     object
 4   Song         69 non-null     object
 5   Points       69 non-null     object
 6                69 non-null     object
 7   link0        69 non-null     object
 8   link1        69 non-null     object
 9   link2        69 non-null     object
 10  link3        69 non-null     object
 11  link4        24 non-null     object
dtypes: object(12)
memory usage: 6.6+ KB


In [ ]:
winners.rename(columns = {'link0':'event_link', 'link1':'country_link', 'link2':'artist_link','link3':'youtube_link'},inplace = True)
winners.drop(['','link4'],axis=1, inplace = True)
winners['event_link'] = base_url + winners['event_link']
winners['country_link'] = base_url + winners['country_link']
winners['artist_link'] = base_url + winners['artist_link']
winners.head()

,Year,City,Winners,Participant,Song,Points,event_link,country_link,artist_link,youtube_link
0,2022,Turin,Ukraine,Kalush Orchestra,Stefania,631,https://eurovision.tv/event/turin-2022,https://eurovision.tv/country/ukraine,https://eurovision.tv/participant/kalush-orche...,https://youtu.be/UiEGVYOruLk
1,2021,Rotterdam,Italy,Måneskin,Zitti E Buoni,524,https://eurovision.tv/event/rotterdam-2021,https://eurovision.tv/country/italy,https://eurovision.tv/participant/maneskin-2021,https://youtu.be/9mL6Cmkg2_A
2,2019,Tel Aviv,Netherlands,Duncan Laurence,Arcade,498,https://eurovision.tv/event/tel-aviv-2019,https://eurovision.tv/country/the-netherlands,https://eurovision.tv/participant/duncan-laurence,https://youtu.be/R3D-r4ogr7s
3,2018,Lisbon,Israel,Netta,TOY,529,https://eurovision.tv/event/lisbon-2018,https://eurovision.tv/country/israel,https://eurovision.tv/participant/netta-barzilai,https://youtu.be/fnB783uNjN0
4,2017,Kyiv,Portugal,Salvador Sobral,Amar Pelos Dois,758,https://eurovision.tv/event/kyiv-2017,https://eurovision.tv/country/portugal,https://eurovision.tv/participant/salvador-sobral,https://youtu.be/ymFVfzu-2mw


In [ ]:
winners.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/winners.csv')

##Generating links for shows

In [ ]:
winners = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/winners.csv', index_col = 0)
events = winners[['Year', 'City', 'event_link']]
events.head()

,Year,City,event_link
0,2022,Turin,https://eurovision.tv/event/turin-2022
1,2021,Rotterdam,https://eurovision.tv/event/rotterdam-2021
2,2019,Tel Aviv,https://eurovision.tv/event/tel-aviv-2019
3,2018,Lisbon,https://eurovision.tv/event/lisbon-2018
4,2017,Kyiv,https://eurovision.tv/event/kyiv-2017


In [ ]:
for i, row in events.iterrows():
  print(row['Year'])

A Semi-Final was introduced in 2004, and a second Semi-Final in 2008. Before it was just one event (final).

* event - city and year, like Turin 2022, 
* show - a final, grand final or semifinal during the event

In [ ]:
#links for each show can be generated by adding the type of the show to the url of the event
events['Year'] = events['Year'].astype('int64')

start_year = events.Year.min() #starting from the beginning
end_year = events.Year.max() + 1

def sf(i):
  if i < 2004: return ['/final']
  if i < 2008: return ['/semi-final','/grand-final']
  return ['/first-semi-final', '/second-semi-final', '/grand-final']

shows_by_year = {i: sf(i) for i in range(start_year, end_year)}

def get_shows_links(year, url):
  links = []
  for s in shows_by_year[year]:
    links.append(url + s)
  return links

shows_links = []
for i, row in events.iterrows():
  shows_links.extend(get_shows_links(row['Year'], row['event_link']))


<ipython-input-110-ab7608db8f9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['Year'] = events['Year'].astype('int64')


In [ ]:
data = [(re.search(r'/([a-z-0-9]+)/[a-z-0-9]+$', link).group(1), re.search(r'[a-z-]+$', link).group(), link ) for link in shows_links]
shows_links = pd.DataFrame(data, columns=['event', 'show', 'link'])

shows_links.head()

,event,show,link
0,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
1,turin-2022,second-semi-final,https://eurovision.tv/event/turin-2022/second-...
2,turin-2022,grand-final,https://eurovision.tv/event/turin-2022/grand-f...
3,rotterdam-2021,first-semi-final,https://eurovision.tv/event/rotterdam-2021/fir...
4,rotterdam-2021,second-semi-final,https://eurovision.tv/event/rotterdam-2021/sec...


In [ ]:
shows_links.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/shows_links.csv')

#Collecting participants info for every year

In [ ]:
shows_links = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/shows_links.csv',index_col = 0)
shows_links.head()

,event,show,link
0,turin-2022,first-semi-final,https://eurovision.tv/event/turin-2022/first-s...
1,turin-2022,second-semi-final,https://eurovision.tv/event/turin-2022/second-...
2,turin-2022,grand-final,https://eurovision.tv/event/turin-2022/grand-f...
3,rotterdam-2021,first-semi-final,https://eurovision.tv/event/rotterdam-2021/fir...
4,rotterdam-2021,second-semi-final,https://eurovision.tv/event/rotterdam-2021/sec...


In [ ]:
shows_links['year'] = shows_links['event'].str.extract(r'([0-9]{4})', expand = False). astype('int64')


In [ ]:
shl_2010_2022 = shows_links[shows_links['year'] >=2010]
shl_2000_2009 = shows_links[(shows_links['year'] >= 2000) & (shows_links['year'] <= 2009)]
shl_1980_1999 = shows_links[(shows_links['year']>= 1980) & (shows_links['year']<= 1999)]
shl_1956_1979 = shows_links[shows_links['year'] <= 1979]

In [ ]:
print(shows_links.shape)
print(shl_2010_2022.shape)
print(shl_2000_2009.shape)
print(shl_1980_1999.shape)
print(shl_1956_1979.shape)

(101, 4)
(36, 4)
(18, 4)
(20, 4)
(27, 4)


In [ ]:


def collect_participants(shows_df, failed_urls):
  all_data = []

  number_of_links = 2 #country, artist
  collect_until = 4 # - youtube links breake the code

  for i, row in shows_df.iterrows():
    r = requests.get(url = row['link'], headers={'User-Agent': 'Mozilla/5.0'})
    if r.ok:
      try:
        df = collect_table(r, '.cols-7', number_of_links, collect_until)
        df['event']  = row['event']
        df['show'] = row['show']
        df['show_link'] = row['link']
        all_data.append(df)
        print(row['event'],row['show'],'collected', sep = ' ')
      except:
        failed_urls.append(row['link'])
    else:
      print(row['link'], r.status_code)
      failed_urls.append(row['link'])
    sleep(random.randrange(2,5,1))
  return pd.concat(all_data, ignore_index=True)


##Collecting data for eahc time period

In [ ]:
failed_urls = []

In [ ]:
participants_1956_1979 = collect_participants(shl_1956_1979, failed_urls)
print(participants_1956_1979.shape)
print(failed_urls)

copenhagen-1964 final collected
london-1963 final collected
luxembourg-1962 final collected
cannes-1961 final collected
london-1960 final collected
cannes-1959 final collected
hilversum-1958 final collected
frankfurt-1957 final collected
lugano-1956 final collected
the-hague-1976 final collected
stockholm-1975 final collected
brighton-1974 final collected
luxembourg-1973 final collected
edinburgh-1972 final collected
dublin-1971 final collected
amsterdam-1970 final collected
madrid-1969 final collected
madrid-1969 final collected
madrid-1969 final collected
madrid-1969 final collected
london-1968 final collected
vienna-1967 final collected
luxembourg-1966 final collected
naples-1965 final collected
jerusalem-1979 final collected
paris-1978 final collected
london-1977 final collected
(432, 12)
[]


In [ ]:
participants_1956_1979.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_1956_1979.csv')

In [ ]:
participants_1980_1999 = collect_participants(shl_1980_1999, failed_urls)
print(participants_1980_1999.shape)
print(failed_urls)

rome-1991 final collected
zagreb-1990 final collected
lausanne-1989 final collected
dublin-1988 final collected
brussels-1987 final collected
bergen-1986 final collected
gothenburg-1985 final collected
luxembourg-1984 final collected
munich-1983 final collected
harrogate-1982 final collected
dublin-1981 final collected
the-hague-1980 final collected
jerusalem-1999 final collected
birmingham-1998 final collected
dublin-1997 final collected
oslo-1996 final collected
dublin-1995 final collected
dublin-1994 final collected
millstreet-1993 final collected
malmo-1992 final collected
(436, 12)
[]


In [ ]:
participants_1980_1999.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_1980_1999.csv')

In [ ]:
participants_2000_2009 = collect_participants(shl_2000_2009, failed_urls)
print(participants_2000_2009.shape)
print(failed_urls)

moscow-2009 first-semi-final collected
moscow-2009 second-semi-final collected
moscow-2009 grand-final collected
belgrade-2008 first-semi-final collected
belgrade-2008 second-semi-final collected
belgrade-2008 grand-final collected
helsinki-2007 semi-final collected
helsinki-2007 grand-final collected
athens-2006 semi-final collected
athens-2006 grand-final collected
kyiv-2005 semi-final collected
kyiv-2005 grand-final collected
istanbul-2004 semi-final collected
istanbul-2004 grand-final collected
riga-2003 final collected
tallinn-2002 final collected
copenhagen-2001 final collected
stockholm-2000 final collected
(417, 12)
[]


In [ ]:

participants_2000_2009.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_2000_2009.csv')

In [ ]:
participants_2010_2022 = collect_participants(shl_2010_2022, failed_urls)
print(participants_2010_2022.shape)
print(failed_urls)

turin-2022 first-semi-final collected
turin-2022 second-semi-final collected
turin-2022 grand-final collected
rotterdam-2021 first-semi-final collected
rotterdam-2021 second-semi-final collected
rotterdam-2021 grand-final collected
tel-aviv-2019 first-semi-final collected
tel-aviv-2019 second-semi-final collected
tel-aviv-2019 grand-final collected
lisbon-2018 first-semi-final collected
lisbon-2018 second-semi-final collected
lisbon-2018 grand-final collected
kyiv-2017 first-semi-final collected
kyiv-2017 second-semi-final collected
kyiv-2017 grand-final collected
stockholm-2016 first-semi-final collected
stockholm-2016 second-semi-final collected
stockholm-2016 grand-final collected
vienna-2015 first-semi-final collected
vienna-2015 second-semi-final collected
vienna-2015 grand-final collected
copenhagen-2014 first-semi-final collected
copenhagen-2014 second-semi-final collected
copenhagen-2014 grand-final collected
malmo-2013 first-semi-final collected
malmo-2013 second-semi-final co

In [ ]:

participants_2010_2022.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_2010_2022.csv')

In [ ]:
participants = pd.concat([participants_1956_1979, participants_1980_1999, participants_2000_2009, participants_2010_2022], ignore_index=True)
participants = participants.rename(columns = {'link0':'country_link', 'link1':'participant_link','R/O\n\nSort descending\n\n\n':'R/O'})
participants['results_link'] = participants['show_link'] + '/results' + participants['country_link'].str.extract(r'(/[a-z\-]+)$',expand=False)
participants['is_qualified'] = participants['Participant'].apply(lambda s: 1 if 'qualified' in s else 0)
participants['Participant'] = participants['Participant'].str.replace('qualified','')
participants['Half'] = participants['Half'].str.replace(r'[^0-9]','', regex=True)
participants['Rank'] = participants['Rank'].str.replace(r'[^0-9]','', regex=True)
participants['country_link'] = base_url + participants['country_link']
participants['participant_link'] = base_url + participants['participant_link']
participants['year'] = participants['event'].str.extract(r'-([0-9]{4})$',expand=False).astype('int32')
participants = participants.replace({'': np.nan})
participants.tail()

,R/O,Half,Country,Participant,Song,Points,Rank,country_link,participant_link,event,show,show_link,results_link,is_qualified,year
2007,21,NaN,Armenia,Eva Rivas,Apricot Stone,141,7,https://eurovision.tv/country/armenia,https://eurovision.tv/participant/eva-rivas,oslo-2010,grand-final,https://eurovision.tv/event/oslo-2010/grand-final,https://eurovision.tv/event/oslo-2010/grand-fi...,0,2010
2008,22,NaN,Germany,Lena,Satellite,246,1,https://eurovision.tv/country/germany,https://eurovision.tv/participant/lena,oslo-2010,grand-final,https://eurovision.tv/event/oslo-2010/grand-final,https://eurovision.tv/event/oslo-2010/grand-fi...,0,2010
2009,23,NaN,Portugal,Filipa Azevedo,Há Dias Assim,43,18,https://eurovision.tv/country/portugal,https://eurovision.tv/participant/filipa-azevedo,oslo-2010,grand-final,https://eurovision.tv/event/oslo-2010/grand-final,https://eurovision.tv/event/oslo-2010/grand-fi...,0,2010
2010,24,NaN,Israel,Harel Skaat,Milim,71,14,https://eurovision.tv/country/israel,https://eurovision.tv/participant/harel-skaat,oslo-2010,grand-final,https://eurovision.tv/event/oslo-2010/grand-final,https://eurovision.tv/event/oslo-2010/grand-fi...,0,2010
2011,25,NaN,Denmark,Chanée & N'evergreen,In A Moment Like This,149,4,https://eurovision.tv/country/denmark,https://eurovision.tv/participant/chanee-n-eve...,oslo-2010,grand-final,https://eurovision.tv/event/oslo-2010/grand-final,https://eurovision.tv/event/oslo-2010/grand-fi...,0,2010


In [ ]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2012 entries, 0 to 2011
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   R/O               2012 non-null   object
 1   Half              73 non-null     object
 2   Country           2012 non-null   object
 3   Participant       2012 non-null   object
 4   Song              2011 non-null   object
 5   Points            1961 non-null   object
 6   Rank              2012 non-null   object
 7   country_link      2012 non-null   object
 8   participant_link  2012 non-null   object
 9   event             2012 non-null   object
 10  show              2012 non-null   object
 11  show_link         2012 non-null   object
 12  results_link      2012 non-null   object
 13  is_qualified      2012 non-null   int64 
 14  year              2012 non-null   int32 
dtypes: int32(1), int64(1), object(13)
memory usage: 228.0+ KB


In [ ]:
participants.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants.csv')

#Collecting votes

In [93]:
participants = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants.csv', index_col = 0)
participants['results_link'] = participants['results_link'].str.replace('/fyr-macedonia', '/north-macedonia').str.replace('/the-netherlands', '/netherlands')

participants.head(2)

,R/O,Half,Country,Participant,Song,Points,Rank,country_link,participant_link,event,show,show_link,results_link,is_qualified,year
0,1,NaN,Luxembourg,Hugues Aufray,Dès Que Le Printemps Revient,14.0,4,https://eurovision.tv/country/luxembourg,https://eurovision.tv/participant/hugues-aufray,copenhagen-1964,final,https://eurovision.tv/event/copenhagen-1964/final,https://eurovision.tv/event/copenhagen-1964/fi...,0,1964
1,2,NaN,Netherlands,Anneke Grönloh,Jij Bent Mijn Leven,2.0,10,https://eurovision.tv/country/the-netherlands,https://eurovision.tv/participant/anneke-groenloh,copenhagen-1964,final,https://eurovision.tv/event/copenhagen-1964/final,https://eurovision.tv/event/copenhagen-1964/fi...,0,1964


In [96]:
results_links = participants[['year','event','show','results_link']].copy().sort_values(by = ['year'], axis=0)
results_links.head()

print(results_links.shape)

(2012, 4)


In [98]:
def get_voting_table_1(soup):
  voted_for = soup.select_one('.details-tab__voted-for-participant')
  participant_name = voted_for.select_one('.participant-name').text.strip()
  voting_tables = voted_for.select('.views-element-container')

  data = []
  for t in voting_tables:
    if t.div:
      table_title = t.div.div.text.strip()
      voting_type = table_title.split()[-1] if 'from' in table_title else 'other'

      for table_row in t.select('.details-tab__voted-row'):
        points = table_row.select_one('div.details-tab__voted-point').text.strip()
        countries = table_row.select('div.details-tab__voter')
        row = []
        for c in countries:
         row.append([voting_type, participant_name, c.text.strip(), points])
        data.extend(row)
  if not data:
    data = [['all_votings', participant_name, 'every_country', 0]]
  columns = ['voting_type', 'points_for', 'points_from', 'points']
  return pd.DataFrame(data=data, columns = columns)
  
    

In [102]:
all_data = []
failed_urls = []
for i, row in results_links.iterrows():
  r = requests.get(url = row['results_link'],headers={'User-Agent': 'Mozilla/5.0'})
  if r.ok:
    try:
      soup = BeautifulSoup(r.text,'html.parser')
      df = get_voting_table_1(soup)
      df['event'] = row['event']
      df['show'] = row['show']
      df['link'] = row['results_link']
      all_data.append(df)
      print(row['results_link'])
    except:
      failed_urls.append(row['results_link'])
      print('failed_url:',row['results_link'])
  else:
    print(r.staus_code, row['results_link'])
  print('failed_urls:', len(failed_urls)) 
  sleep(random.randrange(2,4,1))
votes = pd.concat(all_data, ignore_index=True)
votes.head(30)


failed_url: https://eurovision.tv/event/lugano-1956/final/results/switzerland
failed_urls: 1
failed_url: https://eurovision.tv/event/lugano-1956/final/results/netherlands
failed_urls: 2
failed_url: https://eurovision.tv/event/lugano-1956/final/results/belgium
failed_urls: 3
failed_url: https://eurovision.tv/event/lugano-1956/final/results/germany
failed_urls: 4
failed_url: https://eurovision.tv/event/lugano-1956/final/results/france
failed_urls: 5
failed_url: https://eurovision.tv/event/lugano-1956/final/results/luxembourg
failed_urls: 6
failed_url: https://eurovision.tv/event/lugano-1956/final/results/netherlands
failed_urls: 7
failed_url: https://eurovision.tv/event/lugano-1956/final/results/italy
failed_urls: 8
failed_url: https://eurovision.tv/event/lugano-1956/final/results/belgium
failed_urls: 9
failed_url: https://eurovision.tv/event/lugano-1956/final/results/germany
failed_urls: 10
failed_url: https://eurovision.tv/event/lugano-1956/final/results/france
failed_urls: 11
failed_u

,voting_type,points_for,points_from,points,event,show,link
0,other,Luxembourg,Italy,4,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
1,other,Luxembourg,Austria,3,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
2,other,Luxembourg,United Kingdom,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
3,other,Germany,France,6,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
4,other,Germany,Italy,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
5,other,Germany,Belgium,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
6,other,Belgium,Denmark,2,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
7,other,Belgium,Germany,2,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
8,other,Belgium,Switzerland,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
9,other,United Kingdom,Switzerland,2,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...


In [108]:
votes.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/votes_new.csv')

the only failed urls so far are from the first contest in 1956, its okay, i dont need this information any way

In [110]:
failed_urls

['https://eurovision.tv/event/lugano-1956/final/results/switzerland',
 'https://eurovision.tv/event/lugano-1956/final/results/netherlands',
 'https://eurovision.tv/event/lugano-1956/final/results/belgium',
 'https://eurovision.tv/event/lugano-1956/final/results/germany',
 'https://eurovision.tv/event/lugano-1956/final/results/france',
 'https://eurovision.tv/event/lugano-1956/final/results/luxembourg',
 'https://eurovision.tv/event/lugano-1956/final/results/netherlands',
 'https://eurovision.tv/event/lugano-1956/final/results/italy',
 'https://eurovision.tv/event/lugano-1956/final/results/belgium',
 'https://eurovision.tv/event/lugano-1956/final/results/germany',
 'https://eurovision.tv/event/lugano-1956/final/results/france',
 'https://eurovision.tv/event/lugano-1956/final/results/luxembourg',
 'https://eurovision.tv/event/lugano-1956/final/results/italy',
 'https://eurovision.tv/event/lugano-1956/final/results/switzerland']

I need to write random tests for the votes table

#Checking and cleaning the data, if it's correct, no duplicates, no nulls etc

##Winners data

In [96]:
winners = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/winners.csv', index_col = 0)
winners.head()

,Year,City,Winners,Participant,Song,Points,event_link,country_link,artist_link,youtube_link
0,2022,Turin,Ukraine,Kalush Orchestra,Stefania,631.0,https://eurovision.tv/event/turin-2022,https://eurovision.tv/country/ukraine,https://eurovision.tv/participant/kalush-orche...,https://youtu.be/UiEGVYOruLk
1,2021,Rotterdam,Italy,Måneskin,Zitti E Buoni,524.0,https://eurovision.tv/event/rotterdam-2021,https://eurovision.tv/country/italy,https://eurovision.tv/participant/maneskin-2021,https://youtu.be/9mL6Cmkg2_A
2,2019,Tel Aviv,Netherlands,Duncan Laurence,Arcade,498.0,https://eurovision.tv/event/tel-aviv-2019,https://eurovision.tv/country/the-netherlands,https://eurovision.tv/participant/duncan-laurence,https://youtu.be/R3D-r4ogr7s
3,2018,Lisbon,Israel,Netta,TOY,529.0,https://eurovision.tv/event/lisbon-2018,https://eurovision.tv/country/israel,https://eurovision.tv/participant/netta-barzilai,https://youtu.be/fnB783uNjN0
4,2017,Kyiv,Portugal,Salvador Sobral,Amar Pelos Dois,758.0,https://eurovision.tv/event/kyiv-2017,https://eurovision.tv/country/portugal,https://eurovision.tv/participant/salvador-sobral,https://youtu.be/ymFVfzu-2mw


In [92]:
winners.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 68
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Year          69 non-null     int64  
 1   City          69 non-null     object 
 2   Winners       69 non-null     object 
 3   Participant   69 non-null     object 
 4   Song          24 non-null     object 
 5   Points        68 non-null     float64
 6   event_link    69 non-null     object 
 7   country_link  69 non-null     object 
 8   artist_link   69 non-null     object 
 9   youtube_link  69 non-null     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 5.9+ KB


In [94]:
winners.tail()

,Year,City,Winners,Participant,Song,Points,event_link,country_link,artist_link,youtube_link
64,1996,Oslo,Ireland,Eimear Quinn,NaN,162.0,https://eurovision.tv/event/oslo-1996,https://eurovision.tv/country/ireland,https://eurovision.tv/participant/eimear-quinn,/event/oslo-1996
65,1995,Dublin,Norway,Secret Garden,NaN,148.0,https://eurovision.tv/event/dublin-1995,https://eurovision.tv/country/norway,https://eurovision.tv/participant/secret-garden,/event/dublin-1995
66,1994,Dublin,Ireland,Paul Harrington and Charlie McGettigan,NaN,226.0,https://eurovision.tv/event/dublin-1994,https://eurovision.tv/country/ireland,https://eurovision.tv/participant/paul-harring...,/event/dublin-1994
67,1993,Millstreet,Ireland,Niamh Kavanagh,NaN,187.0,https://eurovision.tv/event/millstreet-1993,https://eurovision.tv/country/ireland,https://eurovision.tv/participant/niamh-kavanagh,/event/millstreet-1993
68,1992,Malmö,Ireland,Linda Martin,NaN,155.0,https://eurovision.tv/event/malmo-1992,https://eurovision.tv/country/ireland,https://eurovision.tv/participant/linda-martin-1,/event/malmo-1992


a lot odf songs is missing, no data about points for the first contest

In [97]:
winners.rename(columns = {'Year':'year','Winners':'country','Participant':'participant', 'Points':'points'}, inplace = True)
winners.drop(['Song', 'youtube_link','City'], axis = 1, inplace=True)
winners['event'] = winners['event_link'].str.extract(r'/([a-z-]+[0-9]{4})$')
winners.head()

,year,country,participant,points,event_link,country_link,artist_link,event
0,2022,Ukraine,Kalush Orchestra,631.0,https://eurovision.tv/event/turin-2022,https://eurovision.tv/country/ukraine,https://eurovision.tv/participant/kalush-orche...,turin-2022
1,2021,Italy,Måneskin,524.0,https://eurovision.tv/event/rotterdam-2021,https://eurovision.tv/country/italy,https://eurovision.tv/participant/maneskin-2021,rotterdam-2021
2,2019,Netherlands,Duncan Laurence,498.0,https://eurovision.tv/event/tel-aviv-2019,https://eurovision.tv/country/the-netherlands,https://eurovision.tv/participant/duncan-laurence,tel-aviv-2019
3,2018,Israel,Netta,529.0,https://eurovision.tv/event/lisbon-2018,https://eurovision.tv/country/israel,https://eurovision.tv/participant/netta-barzilai,lisbon-2018
4,2017,Portugal,Salvador Sobral,758.0,https://eurovision.tv/event/kyiv-2017,https://eurovision.tv/country/portugal,https://eurovision.tv/participant/salvador-sobral,kyiv-2017


In [98]:
winners = winners[['year', 'event', 'country', 'participant','points','event_link', 'country_link']]

In [ ]:
winners.info()

In [101]:
winners[winners.duplicated()]

,year,event,country,participant,points,event_link,country_link


In [103]:
winners.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/winners.csv')

##Participants

In [104]:
p1 = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_2010_2022.csv', index_col = 0)
p2 = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_2000_2009.csv', index_col = 0)
p3 = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_1980_1999.csv', index_col = 0)
p4 = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants_1956_1979.csv', index_col = 0)

In [106]:
participants = pd.concat([p1, p2, p3, p4], ignore_index=True)
participants = participants.rename(columns = {'link0':'country_link', 'link1':'participant_link','R/O\n\nSort descending\n\n\n':'R/O'})
participants['results_link'] = participants['show_link'] + '/results' + participants['country_link'].str.extract(r'(/[a-z\-]+)$',expand=False)
participants['is_qualified'] = participants['Participant'].apply(lambda s: 1 if 'qualified' in s else 0)
participants['Participant'] = participants['Participant'].str.replace('qualified','')
participants['Half'] = participants['Half'].str.replace(r'[^0-9]','', regex=True)
participants['Rank'] = participants['Rank'].str.replace(r'[^0-9]','', regex=True)
participants['country_link'] = base_url + participants['country_link']
participants['participant_link'] = base_url + participants['participant_link']
participants['year'] = participants['event'].str.extract(r'-([0-9]{4})$',expand=False).astype('int32')
participants = participants.replace({'': np.nan})
participants.tail()

,R/O,Half,Country,Participant,Song,Points,Rank,country_link,participant_link,event,show,show_link,results_link,is_qualified,year
2007,14,NaN,Spain,Micky,Enseñame A Cantar,52.0,9,https://eurovision.tv/country/spain,https://eurovision.tv/participant/micky,london-1977,final,https://eurovision.tv/event/london-1977/final,https://eurovision.tv/event/london-1977/final/...,0,1977
2008,15,NaN,Italy,Mia Martini,Liberà,33.0,13,https://eurovision.tv/country/italy,https://eurovision.tv/participant/mia-martini,london-1977,final,https://eurovision.tv/event/london-1977/final,https://eurovision.tv/event/london-1977/final/...,0,1977
2009,16,NaN,Finland,Monica Aspelund,Lapponia,50.0,10,https://eurovision.tv/country/finland,https://eurovision.tv/participant/monica-aspelund,london-1977,final,https://eurovision.tv/event/london-1977/final,https://eurovision.tv/event/london-1977/final/...,0,1977
2010,17,NaN,Belgium,Dream Express,"A Million In One, Two, Three",69.0,7,https://eurovision.tv/country/belgium,https://eurovision.tv/participant/dream-express,london-1977,final,https://eurovision.tv/event/london-1977/final,https://eurovision.tv/event/london-1977/final/...,0,1977
2011,18,NaN,France,Marie Myriam,L'oiseau Et L'enfant,136.0,1,https://eurovision.tv/country/france,https://eurovision.tv/participant/marie-myriam,london-1977,final,https://eurovision.tv/event/london-1977/final,https://eurovision.tv/event/london-1977/final/...,0,1977


In [107]:
participants['results_link'] = participants['results_link'].str.replace('/fyr-macedonia', '/north-macedonia').str.replace('/the-netherlands', '/netherlands')

In [109]:
participants[participants.duplicated()]

,R/O,Half,Country,Participant,Song,Points,Rank,country_link,participant_link,event,show,show_link,results_link,is_qualified,year
1837,1,NaN,Yugoslavia,Ivan,Pozdrav Svijetu,5.0,13,https://eurovision.tv/country/yugoslavia,https://eurovision.tv/participant/ivan,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1838,2,NaN,Luxembourg,Romuald,Cathérine,7.0,11,https://eurovision.tv/country/luxembourg,https://eurovision.tv/participant/romuald-1,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1839,3,NaN,Spain,Salomé,Vivo Cantando,18.0,1,https://eurovision.tv/country/spain,https://eurovision.tv/participant/salome,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1840,4,NaN,Monaco,Jean-Jacques,"Maman, Maman",11.0,6,https://eurovision.tv/country/monaco,https://eurovision.tv/participant/jean-jacques,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1841,5,NaN,Ireland,Muriel Day and the Lindsays,The Wages Of Love,10.0,7,https://eurovision.tv/country/ireland,https://eurovision.tv/participant/muriel-day-a...,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1842,6,NaN,Italy,Iva Zanicchi,Due Grosse Lacrime Bianche,5.0,13,https://eurovision.tv/country/italy,https://eurovision.tv/participant/iva-zanicchi,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1843,7,NaN,United Kingdom,Lulu,Boom Bang-a-bang,18.0,1,https://eurovision.tv/country/united-kingdom,https://eurovision.tv/participant/lulu,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1844,8,NaN,Netherlands,Lenny Kuhr,De Troubadour,18.0,1,https://eurovision.tv/country/the-netherlands,https://eurovision.tv/participant/lenny-kuhr,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1845,9,NaN,Sweden,Tommy Körberg,"Judy, Min Vän",8.0,9,https://eurovision.tv/country/sweden,https://eurovision.tv/participant/tommy-koerberg,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969
1846,10,NaN,Belgium,Louis Neefs,Jennifer Jennings,10.0,7,https://eurovision.tv/country/belgium,https://eurovision.tv/participant/louis-neefs-1,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final,https://eurovision.tv/event/madrid-1969/final/...,0,1969


In [111]:
participants.drop_duplicates(inplace = True)

In [112]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2011
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   R/O               1964 non-null   int64  
 1   Half              73 non-null     object 
 2   Country           1964 non-null   object 
 3   Participant       1964 non-null   object 
 4   Song              1963 non-null   object 
 5   Points            1913 non-null   float64
 6   Rank              1964 non-null   object 
 7   country_link      1964 non-null   object 
 8   participant_link  1964 non-null   object 
 9   event             1964 non-null   object 
 10  show              1964 non-null   object 
 11  show_link         1964 non-null   object 
 12  results_link      1964 non-null   object 
 13  is_qualified      1964 non-null   int64  
 14  year              1964 non-null   int32  
dtypes: float64(1), int32(1), int64(2), object(11)
memory usage: 237.8+ KB


In [113]:
participants.drop('Half', axis = 1, inplace = True)

In [117]:
participants.rename(columns = {'R/O' : 'running_order', 'Country':'country','Participant':'participant','Song':'song','Points':'points','Rank':'rank'}, inplace = True)
participants = participants[['year', 'event', 'show', 'country', 'running_order', 'participant','song', 'points', 'rank', 'is_qualified', 'country_link','participant_link', 'show_link', 'results_link']]

In [121]:
participants['rank'] = pd.to_numeric(participants['rank'])

In [124]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2011
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              1964 non-null   int32  
 1   event             1964 non-null   object 
 2   show              1964 non-null   object 
 3   country           1964 non-null   object 
 4   running_order     1964 non-null   int64  
 5   participant       1964 non-null   object 
 6   song              1963 non-null   object 
 7   points            1913 non-null   float64
 8   rank              1964 non-null   int64  
 9   is_qualified      1964 non-null   int64  
 10  country_link      1964 non-null   object 
 11  participant_link  1964 non-null   object 
 12  show_link         1964 non-null   object 
 13  results_link      1964 non-null   object 
dtypes: float64(1), int32(1), int64(3), object(9)
memory usage: 222.5+ KB


In [118]:
participants.head()

,year,event,show,country,running_order,participant,song,points,rank,is_qualified,country_link,participant_link,show_link,results_link
0,2022,turin-2022,first-semi-final,Albania,1,Ronela Hajati,Sekret,58.0,12,0,https://eurovision.tv/country/albania,https://eurovision.tv/participant/ronela-hajat...,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
1,2022,turin-2022,first-semi-final,Latvia,2,Citi Zēni,Eat Your Salad,55.0,14,0,https://eurovision.tv/country/latvia,https://eurovision.tv/participant/citi-zeni-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
2,2022,turin-2022,first-semi-final,Lithuania,3,Monika Liu,Sentimentai,159.0,7,1,https://eurovision.tv/country/lithuania,https://eurovision.tv/participant/monika-liu-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
3,2022,turin-2022,first-semi-final,Switzerland,4,Marius Bear,Boys Do Cry,118.0,9,1,https://eurovision.tv/country/switzerland,https://eurovision.tv/participant/marius-bear-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
4,2022,turin-2022,first-semi-final,Slovenia,5,LPS,Disko,15.0,17,0,https://eurovision.tv/country/slovenia,https://eurovision.tv/participant/lps-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...


In [123]:
participants.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants.csv')

## Votes

In [5]:
votes = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/votes_new.csv',index_col = 0)
votes.head()

,voting_type,points_for,points_from,points,event,show,link
0,other,Luxembourg,Italy,4,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
1,other,Luxembourg,Austria,3,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
2,other,Luxembourg,United Kingdom,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
3,other,Germany,France,6,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...
4,other,Germany,Italy,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...


In [6]:
votes['year'] = votes.event.str.extract(r'([0-9]{4})$', expand=False).astype('int32')
votes.head()

,voting_type,points_for,points_from,points,event,show,link,year
0,other,Luxembourg,Italy,4,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957
1,other,Luxembourg,Austria,3,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957
2,other,Luxembourg,United Kingdom,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957
3,other,Germany,France,6,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957
4,other,Germany,Italy,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957


In [7]:
votes = votes.replace({'Bosnia &amp; Herzegovina': 'Bosnia & Herzegovina', 'Serbia &amp; Montenegro': 'Serbia & Montenegro'})

In [8]:
votes[votes.duplicated()]

,voting_type,points_for,points_from,points,event,show,link,year
847,other,Spain,Germany,3,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
848,other,Spain,Belgium,3,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
849,other,Spain,Monaco,3,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
850,other,Spain,Luxembourg,2,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
851,other,Spain,Portugal,2,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
...,...,...,...,...,...,...,...,...
1111,other,Spain,Portugal,2,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
1112,other,Spain,France,2,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
1113,other,Spain,Yugoslavia,1,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969
1114,other,Spain,Norway,1,madrid-1969,final,https://eurovision.tv/event/madrid-1969/final/...,1969


In [9]:
votes.drop_duplicates(inplace = True)

In [10]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27637 entries, 0 to 27900
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   voting_type  27637 non-null  object
 1   points_for   27637 non-null  object
 2   points_from  27637 non-null  object
 3   points       27637 non-null  int64 
 4   event        27637 non-null  object
 5   show         27637 non-null  object
 6   link         27637 non-null  object
 7   year         27637 non-null  int32 
dtypes: int32(1), int64(1), object(6)
memory usage: 1.8+ MB


### Cheking if the country from 'points for' is the same in the url 
(if url is broken there will be the alphabeticaly first country in the table)

In [11]:
votes['correct_country'] = votes.apply(lambda x: x.points_for.split()[-1].lower() in x.link, axis = 1)
not_correct_votes = votes[~votes.correct_country]
not_correct_votes= not_correct_votes[['year','show','points_for','link']]

not_correct_votes

,year,show,points_for,link
2316,1975,final,Türkiye,https://eurovision.tv/event/stockholm-1975/fin...
2701,1978,final,Türkiye,https://eurovision.tv/event/paris-1978/final/r...
2702,1978,final,Türkiye,https://eurovision.tv/event/paris-1978/final/r...
3158,1980,final,Türkiye,https://eurovision.tv/event/the-hague-1980/fin...
3159,1980,final,Türkiye,https://eurovision.tv/event/the-hague-1980/fin...
...,...,...,...,...
15474,2012,grand-final,Türkiye,https://eurovision.tv/event/baku-2012/grand-fi...
15475,2012,grand-final,Türkiye,https://eurovision.tv/event/baku-2012/grand-fi...
15476,2012,grand-final,Türkiye,https://eurovision.tv/event/baku-2012/grand-fi...
15477,2012,grand-final,Türkiye,https://eurovision.tv/event/baku-2012/grand-fi...


Its only turkiye, and its because of ü, data itself is correct

In [13]:
not_correct_votes[not_correct_votes.points_for != 'Türkiye']

,year,show,points_for,link


###Comparing votes and participants data


In [14]:
participants = pd.read_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants.csv',index_col = 0)
participants.head()

,year,event,show,country,running_order,participant,song,points,rank,is_qualified,country_link,participant_link,show_link,results_link
0,2022,turin-2022,first-semi-final,Albania,1,Ronela Hajati,Sekret,58.0,12,0,https://eurovision.tv/country/albania,https://eurovision.tv/participant/ronela-hajat...,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
1,2022,turin-2022,first-semi-final,Latvia,2,Citi Zēni,Eat Your Salad,55.0,14,0,https://eurovision.tv/country/latvia,https://eurovision.tv/participant/citi-zeni-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
2,2022,turin-2022,first-semi-final,Lithuania,3,Monika Liu,Sentimentai,159.0,7,1,https://eurovision.tv/country/lithuania,https://eurovision.tv/participant/monika-liu-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
3,2022,turin-2022,first-semi-final,Switzerland,4,Marius Bear,Boys Do Cry,118.0,9,1,https://eurovision.tv/country/switzerland,https://eurovision.tv/participant/marius-bear-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
4,2022,turin-2022,first-semi-final,Slovenia,5,LPS,Disko,15.0,17,0,https://eurovision.tv/country/slovenia,https://eurovision.tv/participant/lps-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...


In [15]:
part_to_check = participants[['year','event', 'show','country','points', 'results_link']]
part_to_check = part_to_check[part_to_check.year > 1956] #i dont have data about 1956
part_to_check

,year,event,show,country,points,results_link
0,2022,turin-2022,first-semi-final,Albania,58.0,https://eurovision.tv/event/turin-2022/first-s...
1,2022,turin-2022,first-semi-final,Latvia,55.0,https://eurovision.tv/event/turin-2022/first-s...
2,2022,turin-2022,first-semi-final,Lithuania,159.0,https://eurovision.tv/event/turin-2022/first-s...
3,2022,turin-2022,first-semi-final,Switzerland,118.0,https://eurovision.tv/event/turin-2022/first-s...
4,2022,turin-2022,first-semi-final,Slovenia,15.0,https://eurovision.tv/event/turin-2022/first-s...
...,...,...,...,...,...,...
2007,1977,london-1977,final,Spain,52.0,https://eurovision.tv/event/london-1977/final/...
2008,1977,london-1977,final,Italy,33.0,https://eurovision.tv/event/london-1977/final/...
2009,1977,london-1977,final,Finland,50.0,https://eurovision.tv/event/london-1977/final/...
2010,1977,london-1977,final,Belgium,69.0,https://eurovision.tv/event/london-1977/final/...


In [16]:
votes_to_check = votes.groupby(['year','event','show','points_for']).points.sum().reset_index()
votes_to_check

,year,event,show,points_for,points
0,1957,frankfurt-1957,final,Austria,3
1,1957,frankfurt-1957,final,Belgium,5
2,1957,frankfurt-1957,final,Denmark,10
3,1957,frankfurt-1957,final,France,17
4,1957,frankfurt-1957,final,Germany,8
...,...,...,...,...,...
1945,2022,turin-2022,second-semi-final,Poland,198
1946,2022,turin-2022,second-semi-final,Romania,118
1947,2022,turin-2022,second-semi-final,San Marino,50
1948,2022,turin-2022,second-semi-final,Serbia,237


In [17]:
merged_df = part_to_check.rename({'year':'year_l', 'event':'event_l', 'show':'show_l', 'points':'points_l'},axis=1).merge(votes_to_check, how='outer', left_on = ['year_l', 'event_l', 'show_l', 'country'], right_on = ['year','event','show','points_for'], indicator= True)
merged_df

,year_l,event_l,show_l,country,points_l,results_link,year,event,show,points_for,points,_merge
0,2022,turin-2022,first-semi-final,Albania,58.0,https://eurovision.tv/event/turin-2022/first-s...,2022,turin-2022,first-semi-final,Albania,58,both
1,2022,turin-2022,first-semi-final,Latvia,55.0,https://eurovision.tv/event/turin-2022/first-s...,2022,turin-2022,first-semi-final,Latvia,55,both
2,2022,turin-2022,first-semi-final,Lithuania,159.0,https://eurovision.tv/event/turin-2022/first-s...,2022,turin-2022,first-semi-final,Lithuania,159,both
3,2022,turin-2022,first-semi-final,Switzerland,118.0,https://eurovision.tv/event/turin-2022/first-s...,2022,turin-2022,first-semi-final,Switzerland,118,both
4,2022,turin-2022,first-semi-final,Slovenia,15.0,https://eurovision.tv/event/turin-2022/first-s...,2022,turin-2022,first-semi-final,Slovenia,15,both
...,...,...,...,...,...,...,...,...,...,...,...,...
1945,1977,london-1977,final,Spain,52.0,https://eurovision.tv/event/london-1977/final/...,1977,london-1977,final,Spain,52,both
1946,1977,london-1977,final,Italy,33.0,https://eurovision.tv/event/london-1977/final/...,1977,london-1977,final,Italy,33,both
1947,1977,london-1977,final,Finland,50.0,https://eurovision.tv/event/london-1977/final/...,1977,london-1977,final,Finland,50,both
1948,1977,london-1977,final,Belgium,69.0,https://eurovision.tv/event/london-1977/final/...,1977,london-1977,final,Belgium,69,both


In [18]:
merged_df[merged_df['_merge'] != 'both']

,year_l,event_l,show_l,country,points_l,results_link,year,event,show,points_for,points,_merge


In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1950 entries, 0 to 1949
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   year_l        1950 non-null   int64   
 1   event_l       1950 non-null   object  
 2   show_l        1950 non-null   object  
 3   country       1950 non-null   object  
 4   points_l      1913 non-null   float64 
 5   results_link  1950 non-null   object  
 6   year          1950 non-null   int64   
 7   event         1950 non-null   object  
 8   show          1950 non-null   object  
 9   points_for    1950 non-null   object  
 10  points        1950 non-null   int64   
 11  _merge        1950 non-null   category
dtypes: category(1), float64(1), int64(3), object(7)
memory usage: 184.8+ KB


In [20]:
merged_df[merged_df.points_l.isnull()]

,year_l,event_l,show_l,country,points_l,results_link,year,event,show,points_for,points,_merge
728,2009,moscow-2009,first-semi-final,Czechia,NaN,https://eurovision.tv/event/moscow-2009/first-...,2009,moscow-2009,first-semi-final,Czechia,0,both
927,2006,athens-2006,semi-final,Serbia & Montenegro,NaN,https://eurovision.tv/event/athens-2006/semi-f...,2006,athens-2006,semi-final,Serbia & Montenegro,0,both
1003,2004,istanbul-2004,semi-final,Switzerland,NaN,https://eurovision.tv/event/istanbul-2004/semi...,2004,istanbul-2004,semi-final,Switzerland,0,both
1061,2003,riga-2003,final,United Kingdom,NaN,https://eurovision.tv/event/riga-2003/final/re...,2003,riga-2003,final,United Kingdom,0,both
1149,1991,rome-1991,final,Austria,NaN,https://eurovision.tv/event/rome-1991/final/re...,1991,rome-1991,final,Austria,0,both
1207,1989,lausanne-1989,final,Iceland,NaN,https://eurovision.tv/event/lausanne-1989/fina...,1989,lausanne-1989,final,Iceland,0,both
1221,1988,dublin-1988,final,Austria,NaN,https://eurovision.tv/event/dublin-1988/final/...,1988,dublin-1988,final,Austria,0,both
1240,1987,brussels-1987,final,Türkiye,NaN,https://eurovision.tv/event/brussels-1987/fina...,1987,brussels-1987,final,Türkiye,0,both
1316,1983,munich-1983,final,Türkiye,NaN,https://eurovision.tv/event/munich-1983/final/...,1983,munich-1983,final,Türkiye,0,both
1317,1983,munich-1983,final,Spain,NaN,https://eurovision.tv/event/munich-1983/final/...,1983,munich-1983,final,Spain,0,both


In [21]:
participants['points'] = participants['points'].fillna(value = 0)
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2011
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              1964 non-null   int64  
 1   event             1964 non-null   object 
 2   show              1964 non-null   object 
 3   country           1964 non-null   object 
 4   running_order     1964 non-null   int64  
 5   participant       1964 non-null   object 
 6   song              1963 non-null   object 
 7   points            1964 non-null   float64
 8   rank              1964 non-null   int64  
 9   is_qualified      1964 non-null   int64  
 10  country_link      1964 non-null   object 
 11  participant_link  1964 non-null   object 
 12  show_link         1964 non-null   object 
 13  results_link      1964 non-null   object 
dtypes: float64(1), int64(4), object(9)
memory usage: 230.2+ KB


In [25]:
merged_df[(~merged_df.points_l.isnull()) & (merged_df.points_l != merged_df.points)]

,year_l,event_l,show_l,country,points_l,results_link,year,event,show,points_for,points,_merge


In [26]:
participants.head()

,year,event,show,country,running_order,participant,song,points,rank,is_qualified,country_link,participant_link,show_link,results_link
0,2022,turin-2022,first-semi-final,Albania,1,Ronela Hajati,Sekret,58.0,12,0,https://eurovision.tv/country/albania,https://eurovision.tv/participant/ronela-hajat...,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
1,2022,turin-2022,first-semi-final,Latvia,2,Citi Zēni,Eat Your Salad,55.0,14,0,https://eurovision.tv/country/latvia,https://eurovision.tv/participant/citi-zeni-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
2,2022,turin-2022,first-semi-final,Lithuania,3,Monika Liu,Sentimentai,159.0,7,1,https://eurovision.tv/country/lithuania,https://eurovision.tv/participant/monika-liu-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
3,2022,turin-2022,first-semi-final,Switzerland,4,Marius Bear,Boys Do Cry,118.0,9,1,https://eurovision.tv/country/switzerland,https://eurovision.tv/participant/marius-bear-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...
4,2022,turin-2022,first-semi-final,Slovenia,5,LPS,Disko,15.0,17,0,https://eurovision.tv/country/slovenia,https://eurovision.tv/participant/lps-22,https://eurovision.tv/event/turin-2022/first-s...,https://eurovision.tv/event/turin-2022/first-s...


In [27]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1964 entries, 0 to 2011
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              1964 non-null   int64  
 1   event             1964 non-null   object 
 2   show              1964 non-null   object 
 3   country           1964 non-null   object 
 4   running_order     1964 non-null   int64  
 5   participant       1964 non-null   object 
 6   song              1963 non-null   object 
 7   points            1964 non-null   float64
 8   rank              1964 non-null   int64  
 9   is_qualified      1964 non-null   int64  
 10  country_link      1964 non-null   object 
 11  participant_link  1964 non-null   object 
 12  show_link         1964 non-null   object 
 13  results_link      1964 non-null   object 
dtypes: float64(1), int64(4), object(9)
memory usage: 230.2+ KB


In [28]:
votes.head()

,voting_type,points_for,points_from,points,event,show,link,year,correct_country
0,other,Luxembourg,Italy,4,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957,True
1,other,Luxembourg,Austria,3,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957,True
2,other,Luxembourg,United Kingdom,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957,True
3,other,Germany,France,6,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957,True
4,other,Germany,Italy,1,frankfurt-1957,final,https://eurovision.tv/event/frankfurt-1957/fin...,1957,True


In [29]:
votes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27637 entries, 0 to 27900
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   voting_type      27637 non-null  object
 1   points_for       27637 non-null  object
 2   points_from      27637 non-null  object
 3   points           27637 non-null  int64 
 4   event            27637 non-null  object
 5   show             27637 non-null  object
 6   link             27637 non-null  object
 7   year             27637 non-null  int32 
 8   correct_country  27637 non-null  bool  
dtypes: bool(1), int32(1), int64(1), object(6)
memory usage: 1.8+ MB


In [31]:
participants.to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/participants.csv')
votes[['year', 'event','show','points_for', 'points_from','points','voting_type','link']].to_csv('/content/gdrive/MyDrive/Portfolio/Eurovision/votes.csv')

The end of scraping eurovision.tv